In [ ]:
# Plot ua
# Use kernel geomet-ua 
# Smith Dec 2025. Michael.Smith2@nrcan-rncan.gc.ca
# Retrieves and plots observed/fx soundings from UW and ECCC
# Datamart link: https://dd.weather.gc.ca/<YYYYMMDD>/WXO-DD/vertical_profile/
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.plots import SkewT, add_timestamp, Hodograph
from metpy.units import units
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


# To-do:
# Add everything to make the prog tephis from ECCC work
# Figure out why the parcel sounding isn't plotting correctly
# Add calculations to the mix and figure out where to plot them. See https://projectpythia.org/metpy-cookbook/notebooks/skewt/sounding-calculations/
# For some other layout and examples see https://unidata.github.io/MetPy/latest/examples/Advanced_Sounding_With_Complex_Layout.html#sphx-glr-examples-advanced-sounding-with-complex-layout-py 


In [158]:
# User options
# List of stations for fx: https://dd.weather.gc.ca/20251223/WXO-DD/vertical_profile/doc/station_list_for_vertical_profile.txt
# List of stations for obs can be found at: https://weather.uwyo.edu/upperair/sounding_legacy.html 
stn_id = '71964'
date = '20251223'  # YYYYMMDD in UTC
hour = '12'  # HH in UTC
skew_type = 'obs' # 'obs' or 'fx'


# Check on inputs and exit if invalid
if skew_type.lower() not in ['obs', 'fx']:
    raise ValueError("skew_type must be either 'obs' or 'fx'")
if hour not in ['00','06', '12', '18']:
    raise ValueError("hour must be either '00', '06', '12' or '18'")
date_test = pd.to_datetime(date, format='%Y%m%d', utc=True)  # will raise error if invalid
if not (date_test <= pd.Timestamp.utcnow().normalize()):
    raise ValueError("date must be in the past or present (UTC)")

In [ ]:
# Build a URL to grab the csv output from UW 
def get_uw_ua(date, hour, stn_id):

    # URL format for UW csv observed UA is
    # https://weather.uwyo.edu/wsgi/sounding?datetime=2025-12-23%2012:00:00&id=71964&type=TEXT:CSV
    date_formatted = f'{date[0:4]}-{date[4:6]}-{date[6:8]}'
    url = f'https://weather.uwyo.edu/wsgi/sounding?datetime={date_formatted}%20{hour}:00:00&id={stn_id}&type=TEXT:CSV'
    return url

# Some functions that will be called to manipulate and plot the data
# Some code is from the MetPy Cookbook: https://unidata.github.io/MetPy/latest/examples/skewt_soundings/Skew-T_Soundings.html
def plot_skewt(df):

    # Convert non-numeric data to NaN in key columns
    key_cols = ['pressure_hPa', 'temperature_C', 'dew point temperature_C', 'wind speed_m/s', 'wind direction_degree']
    for col in key_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    pres = df['pressure_hPa'].values * units.hPa
    temp = df['temperature_C'].values * units.degC
    dewpoint = df['dew point temperature_C'].values * units.degC
    wind_speed = df['wind speed_m/s'].values.astype(float) * 3.6 * units.km / units.h
    wind_dir = df['wind direction_degree'].values * units.degrees
    u, v = mpcalc.wind_components(wind_speed, wind_dir)

    # Define the figure and rotation
    fig = plt.Figure(figsize=(9, 9))
    skew = SkewT(fig, rotation=45)

    # plot t, td, wind
    skew.plot(pres, temp, 'red')
    skew.plot(pres, dewpoint, 'blue')

    barb_interval = np.arange(100, 1000, 50) * units('hPa')
    ix = mpcalc.resample_nn_1d(pres, barb_interval)
    skew.plot_barbs(pres[ix], u[ix], v[ix], xloc=1)

    # Calculate and plot LCL and parcel profile
    lcl_pressure, lcl_temperature = mpcalc.lcl(pres[0], temp[0], dewpoint[0])
    skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')

    profile = mpcalc.parcel_profile(pres, temp[1], dewpoint[1]).to('degC')
    skew.plot(pres[1:], profile[1:], 'k', linestyle='dashed', linewidth=2)

    # Tweak the labels and axes
    skew.ax.set_xlabel('Temperature (°C)')
    skew.ax.set_ylabel('Pressure (hPa)')
    skew.ax.set_ylim(1000, 100)
    skew.ax.set_xlim(-55, 30)

    skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

    # Plot adiabats and mixing lines
    skew.plot_dry_adiabats(t0=np.arange(233, 533, 15) * units.K, linewidth=0.5, alpha=0.05, color='orangered')
    skew.plot_moist_adiabats(t0=np.arange(233, 400, 10) * units.K, linewidth=0.5, alpha=0.05, color='tab:green')
    skew.plot_mixing_lines(pressure=np.arange(1000, 99, -25) * units.hPa, linewidth=0.5, linestyle='dotted', color='tab:blue')

    # Add geopotential height labels on the primary y-axis
    target_pressures = [1000, 850, 700, 500, 300, 200, 100]
    pres_df = df['pressure_hPa'].dropna().values
    height_df = df['geopotential height_m'].dropna().values
    if len(pres_df) > 1 and len(height_df) > 1:
        for p in target_pressures:
            if p >= pres_df.min() and p <= pres_df.max():
                h = np.interp(p, pres_df[::-1], height_df[::-1])
                h_dm = h / 10  # Convert to decameters
                skew.ax.text(-50, p, f'{h_dm:.0f} dm', fontsize=9, color='gray', ha='left', va='center')

    
    # Add a hodograph to the top left
    ax_hod = inset_axes(skew.ax, '35%','35%', loc=1)
    h = Hodograph(ax_hod, component_range=80)
    h.add_grid(increment=20)
    h.plot_colormapped(u, v, pres, cmap='viridis')
    
    
    return skew


# Function to create a name for the output figure
def make_title(type, site, date, hour):
    return f'{site}_{type}_{date}__{hour}UTC_skewT.png'

In [160]:
# Main
if __name__ == '__main__':
    from datetime import datetime

    # module to pull and format data from University of Wyoming for observed soundings, and datamart for progs
    # The format of the ECCC observed soundings is bad, and it also drops a lot of wind data.
    if skew_type.lower() == 'obs':
        url = get_uw_ua(date, hour, stn_id)
        df = pd.read_csv(url, sep=',', header=0)
    elif skew_type.lower() == 'fx':
        url = f'https://dd.weather.gc.ca/{date}/WXO-DD/vertical_profile/forecast/csv/ProgTephi_{hour}_{stn_id}.csv'
        df = pd.read_csv(url, header = 1).drop(0).reset_index(drop=True)
    
    
    skewt = plot_skewt(df)

    skewt.ax.set_title(f'{skew_type.upper()} Skew-T for Station {stn_id} valid {date} {hour}UTC', fontsize='large')
    skewt.ax.set_adjustable

    current_utc = pd.Timestamp.utcnow()
    add_timestamp(skewt.ax, time=current_utc, y=-0.10, x=0.0, ha='left', time_format='%Y-%m-%d %H:%M UTC', fontsize='medium')

    # Add label for secondary y-axis (height)
    skewt.ax.text(1.08, 0.5, 'Wind (km/h)', transform=skewt.ax.transAxes, rotation=90, va='center', ha='left', fontsize='medium')

    skewt.ax.figure.savefig(make_title(skew_type, stn_id, date, hour))

/tmp/ipykernel_382021/126286030.py:43: UserWarning: Duplicate pressure(s) [7.1 7.2 7.3 7.4 7.5 7.6 7.7 7.8 7.9 8.0 8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8 8.9 9.0 9.1 9.2 9.3 9.4 9.5 9.6 9.7 9.8 9.9 10.0 10.1 10.2 10.3 10.4 10.5 10.6 10.7 10.8 10.9 11.0 11.1 11.2 11.3 11.4 11.5 11.6 11.7 11.8 11.9 12.0 12.1 12.2 12.3 12.4 12.5 12.6 12.7 12.8 12.9 13.0 13.1 13.2 13.3 13.4 13.5 13.6 13.7 13.8 13.9 14.0 14.1 14.2 14.3 14.4 14.5 14.6 14.7 14.8 14.9 15.0 15.1 15.2 15.3 15.4 15.5 15.6 15.7 15.8 15.9 16.0 16.1 16.2 16.3 16.4 16.5 16.6 16.7 16.8 16.9 17.0 17.1 17.2 17.3 17.4 17.5 17.6 17.7 17.8 17.9 18.0 18.1 18.2 18.3 18.4 18.5 18.6 18.7 18.8 18.9 19.0 19.1 19.2 19.3 19.4 19.5 19.6 19.7 19.8 19.9 20.0 20.1 20.2 20.3 20.4 20.5 20.6 20.7 20.8 20.9 21.0 21.1 21.2 21.3 21.4 21.5 21.6 21.7 21.8 21.9 22.0 22.1 22.2 22.3 22.4 22.5 22.6 22.7 22.8 22.9 23.0 23.1 23.2 23.3 23.4 23.5 23.6 23.7 23.8 23.9 24.0 24.1 24.2 24.3 24.4 24.5 24.6 24.7 24.8 24.9 25.0 25.1 25.2 25.3 25.4 25.5 25.6 25.7 25.8 25.9 26.0 26.1